In [59]:
import requests
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [60]:
r= requests.get("https://data.heatonresearch.com/data/t81-558/text/"\
                "treasure_island.txt")
raw_text =r.text
raw_text[0:100]

'ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson\r\n\r\nThis eBook is for th'

In [61]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

In [62]:
processed_text[0:100]

'the project gutenberg ebook of treasure island, by robert louis stevenson\r\n\r\nthis ebook is for the u'

In [63]:
chars = sorted(list(set(processed_text)))
chars
char2idx = dict((c,i) for i,c in enumerate(chars))
idx2char = dict((i,c) for i,c in enumerate(chars))

In [64]:
maxlen = 40
step = 5
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 79472


In [65]:
sentences

['the project gutenberg ebook of treasure ',
 'roject gutenberg ebook of treasure islan',
 't gutenberg ebook of treasure island, by',
 'enberg ebook of treasure island, by robe',
 'g ebook of treasure island, by robert lo',
 'ok of treasure island, by robert louis s',
 ' treasure island, by robert louis steven',
 'sure island, by robert louis stevenson\r\n',
 'island, by robert louis stevenson\r\n\r\nthi',
 'd, by robert louis stevenson\r\n\r\nthis ebo',
 ' robert louis stevenson\r\n\r\nthis ebook is',
 'rt louis stevenson\r\n\r\nthis ebook is for ',
 'uis stevenson\r\n\r\nthis ebook is for the u',
 'tevenson\r\n\r\nthis ebook is for the use of',
 'son\r\n\r\nthis ebook is for the use of anyo',
 '\r\nthis ebook is for the use of anyone an',
 's ebook is for the use of anyone anywher',
 'ok is for the use of anyone anywhere at ',
 ' for the use of anyone anywhere at no co',
 'the use of anyone anywhere at no cost an',
 'se of anyone anywhere at no cost and wit',
 ' anyone anywhere at n

In [66]:
len(sentences)

79472

In [67]:
from sklearn.preprocessing import OneHotEncoder

In [68]:
len(chars)

60

In [69]:
ohe = OneHotEncoder()
ohe.fit(np.array(chars).reshape(-1,1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [70]:
y = np.array(ohe.transform(np.array(next_chars).reshape(-1,1)).todense())

In [71]:
final = []
for lis in sentences:
  split = [i for i in lis]
  final.append(split)

In [72]:
final = np.array(final)

In [73]:
final.shape

(79472, 40)

In [74]:
final_list = []
for lis in final:
  one_hot = ohe.transform(lis.reshape(-1,1))
  final_list.append(np.array(one_hot.todense()))

In [75]:
len(final_list)

79472

In [76]:
 x = np.array(final_list)

In [77]:
x.shape

(79472, 40, 60)

In [104]:
model = tf.keras.Sequential([
tf.keras.layers.LSTM(128, input_shape=(40, len(chars))),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Dense(len(chars), activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])


In [105]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               96768     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


In [106]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [107]:
model.fit(x,y,batch_size=128,epochs=50,validation_split=0.2,callbacks=[callback1])

Epoch 1/50
497/497 [==============================] - 5s 8ms/step - loss: 2.6467 - accuracy: 0.2724 - val_loss: 2.0963 - val_accuracy: 0.3993
Epoch 2/50
497/497 [==============================] - 3s 6ms/step - loss: 1.9421 - accuracy: 0.4353 - val_loss: 1.9513 - val_accuracy: 0.4359
Epoch 3/50
497/497 [==============================] - 3s 6ms/step - loss: 1.7696 - accuracy: 0.4787 - val_loss: 1.8883 - val_accuracy: 0.4535
Epoch 4/50
497/497 [==============================] - 3s 6ms/step - loss: 1.6857 - accuracy: 0.4993 - val_loss: 1.8545 - val_accuracy: 0.4675
Epoch 5/50
497/497 [==============================] - 3s 6ms/step - loss: 1.6016 - accuracy: 0.5188 - val_loss: 1.8187 - val_accuracy: 0.4763
Epoch 6/50
497/497 [==============================] - 3s 6ms/step - loss: 1.5632 - accuracy: 0.5307 - val_loss: 1.8376 - val_accuracy: 0.4774
Epoch 7/50
497/497 [==============================] - 3s 6ms/step - loss: 1.5062 - accuracy: 0.5450 - val_loss: 1.8237 - val_accuracy: 0.4802
Epoch 

In [108]:
idx2char[np.argmax(model.predict(tf.expand_dims(x[0],axis=0)))]

'i'

In [109]:
example = processed_text[150:190]

In [110]:
target_text = processed_text[150:490]

In [111]:
example

'no restrictions whatsoever.  you may cop'

In [112]:
 generated = ''
 generated +=example
 for i in range(400):
    example = np.array([i for i in example])
    one_hot = np.array(ohe.transform(example.reshape(-1,1)).todense())
    preds = model.predict(tf.expand_dims(one_hot,axis=0), verbose=0)[0]
    next_index = np.random.choice(len(preds), p=preds)
    # next_index = np.argmax(preds
    next_char = idx2char[next_index]
    generated += next_char
    example = "".join(example)
    example = example[1:] + next_char

In [113]:
generated

"no restrictions whatsoever.  you may copged for the woud fill usking and a preaivessed as with pessaces\r\nas ha fire compling with i cang pained his poor you fere at the bip was a may but purpayed sefting and bole, and which mi, it was tastany on in there's lieg get by parring\r\nat the shoulder pertune aid\r\nshe took molly sir! alate appearow. yrough frewing-no in the mocking,, and to a mait it we mound the score, and\r\nslicely in which the"

In [114]:
target_text

'no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.net\r\n\r\n\r\ntitle: treasure island\r\n\r\nauthor: robert louis stevenson\r\n\r\nillustrator: milo winter\r\n\r\nrelease date: january 12, 2009 [ebook #27780]\r\n\r\nlanguage: english\r\n'